In [ ]:
# ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำภาษาไทย
# บริษัท วี89 เทคโนโลยี จำกัด
# Enhanced Thai Medical Care AI Agent Chat with CoT, v2.0 - With typhoon2.1-gemma3-4b, Vision, Audio

print("🚀 Installing required packages...")

# 📦 Install required packages for Python 3.12.11 compatibility
!pip install -q --upgrade pip==24.0
!pip install -q --force-reinstall torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124
!pip install -q compressed-tensors>=0.7.0
!pip install -q transformers==4.50.0 accelerate==1.1.0 bitsandbytes==0.47.0
!pip install -q numpy==2.0.2 scipy==1.14.1 fsspec==2025.3.2 rich==12.4.4 pandas==2.2.2
!pip install -q gradio pythainlp sentence-transformers
!pip install -q  datasets==2.20.0 evaluate==0.4.1 rouge-score==0.1.2
!pip install -q openai-whisper pillow timm
!pip install -q sqlalchemy cryptography bcrypt
!pip install -q opencv-python-headless>=4.9.0.80 librosa soundfile
!pip install -q huggingface_hub
!pip install -q matplotlib
#!pip install -q flash-attn --no-build-isolation

print("📦 Packages installed successfully!")

import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    ViTImageProcessor,
    ViTForImageClassification,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    AutoProcessor,
    AutoImageProcessor,
    AutoModelForImageClassification,
)
import json
import re
import os
import hashlib
import sqlite3
import pickle
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict
from enum import Enum
import warnings
import numpy as np
from PIL import Image
import librosa
import soundfile as sf
from cryptography.fernet import Fernet
import base64
import uuid
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
warnings.filterwarnings('ignore')

# Import Thai NLP libraries
try:
    from pythainlp.tokenize import word_tokenize
    from pythainlp.corpus.common import thai_stopwords
    THAI_NLP_AVAILABLE = True
except:
    THAI_NLP_AVAILABLE = False
    def word_tokenize(text, engine='basic'):
        return text.split()
    def thai_stopwords():
        return set()

In [ ]:
# Mount Google Drive with error handling
def setup_drive_connection():
    """Setup Google Drive connection with fallback"""
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = "/content/drive/MyDrive/V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"
        print("Google Drive connected successfully")
        return drive_path
    except:
        local_path = "V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"
        os.makedirs(local_path, exist_ok=True)
        print("Running outside Colab - using local directory")
        return local_path

drive_path = setup_drive_connection()

# ========================= Enhanced Enums and Data Classes =========================

class MedicalDomain(Enum):
    EMERGENCY = "emergency"
    DIAGNOSIS = "diagnosis"
    TREATMENT = "treatment"
    MEDICATION = "medication"
    PREVENTION = "prevention"
    SYMPTOM_ASSESSMENT = "symptom_assessment"
    IMAGE_ANALYSIS = "image_analysis"
    VOICE_ANALYSIS = "voice_analysis"

class UrgencyLevel(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class ConsentType(Enum):
    DATA_COLLECTION = "data_collection"
    DATA_PROCESSING = "data_processing"
    DATA_SHARING = "data_sharing"
    IMAGE_PROCESSING = "image_processing"
    VOICE_PROCESSING = "voice_processing"

@dataclass
class UserConsent:
    user_id: str
    consent_type: ConsentType
    granted: bool
    timestamp: datetime
    ip_address: str = ""
    details: Dict = field(default_factory=dict)

@dataclass
class UserProfile:
    user_id: str
    created_at: datetime
    name: str = ""
    age: int = 0
    gender: str = ""
    medical_history: List[str] = field(default_factory=list)
    current_medications: List[str] = field(default_factory=list)
    allergies: List[str] = field(default_factory=list)
    emergency_contact: str = ""
    consents: List[UserConsent] = field(default_factory=list)
    encrypted_data: Dict = field(default_factory=dict)

@dataclass
class InteractionFeedback:
    interaction_id: str
    user_id: str
    timestamp: datetime
    rating: int  # 1-5
    helpful: bool
    accurate: bool
    comments: str = ""
    symptoms_resolved: bool = False
    follow_up_needed: bool = False

@dataclass
class ClarificationQuestion:
    question_id: str
    category: str
    question_text: str
    options: List[str]
    required: bool = False
    follow_up_questions: List[str] = field(default_factory=list)

# ========================= PDPA Compliance and Security =========================

class PDPACompliance:
    """Handle PDPA compliance and data protection"""

    def __init__(self):
        self.encryption_key = Fernet.generate_key()
        self.cipher = Fernet(self.encryption_key)
        self.consent_database = {}
        self.audit_log = []

    def encrypt_sensitive_data(self, data: str) -> str:
        """Encrypt sensitive personal data"""
        encrypted = self.cipher.encrypt(data.encode())
        return base64.b64encode(encrypted).decode()

    def decrypt_sensitive_data(self, encrypted_data: str) -> str:
        """Decrypt sensitive personal data"""
        try:
            decoded = base64.b64decode(encrypted_data.encode())
            decrypted = self.cipher.decrypt(decoded)
            return decrypted.decode()
        except:
            return ""

    def request_consent(self, user_id: str, consent_type: ConsentType) -> Dict:
        """Request user consent for data processing"""
        consent_id = str(uuid.uuid4())
        consent_request = {
            "consent_id": consent_id,
            "user_id": user_id,
            "type": consent_type.value,
            "timestamp": datetime.now().isoformat(),
            "message": self._get_consent_message(consent_type),
            "options": ["ยินยอม", "ไม่ยินยอม"]
        }
        return consent_request

    def _get_consent_message(self, consent_type: ConsentType) -> str:
        """Get consent message in Thai"""
        messages = {
            ConsentType.DATA_COLLECTION: "ขออนุญาตเก็บข้อมูลส่วนบุคคลเพื่อการให้บริการทางการแพทย์",
            ConsentType.DATA_PROCESSING: "ขออนุญาตประมวลผลข้อมูลสุขภาพเพื่อการวิเคราะห์และให้คำแนะนำ",
            ConsentType.DATA_SHARING: "ขออนุญาตแบ่งปันข้อมูลกับบุคลากรทางการแพทย์",
            ConsentType.IMAGE_PROCESSING: "ขออนุญาตประมวลผลภาพเพื่อการวิเคราะห์ทางการแพทย์",
            ConsentType.VOICE_PROCESSING: "ขออนุญาตประมวลผลเสียงเพื่อการวิเคราะห์อาการ"
        }
        return messages.get(consent_type, "ขออนุญาตดำเนินการ")

    def record_consent(self, user_consent: UserConsent):
        """Record user consent decision"""
        if user_consent.user_id not in self.consent_database:
            self.consent_database[user_consent.user_id] = []
        self.consent_database[user_consent.user_id].append(user_consent)

        # Add to audit log
        self.audit_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": "consent_recorded",
            "user_id": user_consent.user_id,
            "consent_type": user_consent.consent_type.value,
            "granted": user_consent.granted
        })

    def check_consent(self, user_id: str, consent_type: ConsentType) -> bool:
        """Check if user has given consent"""
        if user_id not in self.consent_database:
            return False

        user_consents = self.consent_database[user_id]
        for consent in reversed(user_consents):  # Check most recent first
            if consent.consent_type == consent_type:
                return consent.granted
        return False

    def anonymize_data(self, data: Dict) -> Dict:
        """Anonymize personal data for storage"""
        anonymized = data.copy()
        sensitive_fields = ['name', 'phone', 'email', 'address', 'id_card']

        for field in sensitive_fields:
            if field in anonymized:
                anonymized[field] = hashlib.sha256(str(anonymized[field]).encode()).hexdigest()[:8]

        return anonymized

# ========================= User Profile Storage System =========================

class UserProfileStorage:
    """Secure user profile storage with PDPA compliance"""

    def __init__(self, db_path: str = "medical_profiles.db"):
        self.db_path = db_path
        self.pdpa = PDPACompliance()
        self.init_database()

    def init_database(self):
        """Initialize SQLite database for user profiles"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Create users table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id TEXT PRIMARY KEY,
                created_at TIMESTAMP,
                encrypted_profile TEXT,
                last_updated TIMESTAMP
            )
        ''')

        # Create interactions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS interactions (
                interaction_id TEXT PRIMARY KEY,
                user_id TEXT,
                timestamp TIMESTAMP,
                interaction_type TEXT,
                encrypted_data TEXT,
                FOREIGN KEY (user_id) REFERENCES users (user_id)
            )
        ''')

        # Create feedback table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS feedback (
                feedback_id TEXT PRIMARY KEY,
                interaction_id TEXT,
                user_id TEXT,
                timestamp TIMESTAMP,
                rating INTEGER,
                helpful BOOLEAN,
                accurate BOOLEAN,
                comments TEXT,
                FOREIGN KEY (interaction_id) REFERENCES interactions (interaction_id)
            )
        ''')

        conn.commit()
        conn.close()

    def create_user_profile(self, user_data: Dict) -> UserProfile:
        """Create a new user profile with consent"""
        user_id = str(uuid.uuid4())
        profile = UserProfile(
            user_id=user_id,
            created_at=datetime.now(),
            name=user_data.get('name', ''),
            age=user_data.get('age', 0),
            gender=user_data.get('gender', ''),
            medical_history=user_data.get('medical_history', []),
            current_medications=user_data.get('medications', []),
            allergies=user_data.get('allergies', []),
            emergency_contact=user_data.get('emergency_contact', '')
        )

        # Encrypt sensitive data
        encrypted_profile = self.pdpa.encrypt_sensitive_data(
            json.dumps(asdict(profile), default=str)
        )

        # Store in database
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO users (user_id, created_at, encrypted_profile, last_updated)
            VALUES (?, ?, ?, ?)
        ''', (user_id, datetime.now(), encrypted_profile, datetime.now()))
        conn.commit()
        conn.close()

        return profile

    def get_user_profile(self, user_id: str) -> Optional[UserProfile]:
        """Retrieve user profile with decryption"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT encrypted_profile FROM users WHERE user_id = ?', (user_id,))
        result = cursor.fetchone()
        conn.close()

        if result:
            decrypted_data = self.pdpa.decrypt_sensitive_data(result[0])
            if decrypted_data:
                profile_dict = json.loads(decrypted_data)
                return UserProfile(**profile_dict)
        return None

    def update_user_profile(self, user_id: str, updates: Dict):
        """Update user profile with audit trail"""
        profile = self.get_user_profile(user_id)
        if profile:
            for key, value in updates.items():
                if hasattr(profile, key):
                    setattr(profile, key, value)

            # Encrypt and save
            encrypted_profile = self.pdpa.encrypt_sensitive_data(
                json.dumps(asdict(profile), default=str)
            )

            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute('''
                UPDATE users SET encrypted_profile = ?, last_updated = ?
                WHERE user_id = ?
            ''', (encrypted_profile, datetime.now(), user_id))
            conn.commit()
            conn.close()

    def store_interaction(self, user_id: str, interaction_data: Dict) -> str:
        """Store user interaction with encryption"""
        interaction_id = str(uuid.uuid4())
        encrypted_data = self.pdpa.encrypt_sensitive_data(json.dumps(interaction_data))

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO interactions (interaction_id, user_id, timestamp, interaction_type, encrypted_data)
            VALUES (?, ?, ?, ?, ?)
        ''', (interaction_id, user_id, datetime.now(), interaction_data.get('type', 'chat'), encrypted_data))
        conn.commit()
        conn.close()

        return interaction_id

    def store_feedback(self, feedback: InteractionFeedback):
        """Store user feedback for improvement"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO feedback (feedback_id, interaction_id, user_id, timestamp, rating, helpful, accurate, comments)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            str(uuid.uuid4()),
            feedback.interaction_id,
            feedback.user_id,
            feedback.timestamp,
            feedback.rating,
            feedback.helpful,
            feedback.accurate,
            feedback.comments
        ))
        conn.commit()
        conn.close()

# ========================= Real-time Emergency Triage System =========================

class EmergencyTriageSystem:
    def __init__(self):
        self.triage_criteria = {
            'critical': ['เจ็บหน้าอก', 'หายใจไม่ออก', 'หมดสติ', 'ชัก'],
            'high': ['ไข้สูง', 'ปวดหัวรุนแรง', 'อาเจียนเป็นเลือด'],
            'medium': ['ปวดท้อง', 'ไข้', 'ผื่น'],
            'low': ['อาการเล็กน้อย', 'คำถามทั่วไป']
        }

    def assess_urgency(self, text, image_analysis=None):
        score = 0
        text_lower = text.lower()

        for keyword in self.triage_criteria['critical']:
            if keyword in text_lower:
                return 'critical'

        # Advanced scoring logic
        if image_analysis and 'abnormal' in image_analysis.get('prediction', '').lower():
            score += 2

        return 'high' if score >= 2 else 'medium' if score >= 1 else 'low'

# ========================= Medication Interaction Checker =========================

class DrugInteractionChecker:
    def __init__(self):
        self.interaction_db = {
            'warfarin': ['aspirin', 'nsaids', 'antibiotics'],
            'metformin': ['contrast', 'alcohol'],
            'ace_inhibitors': ['potassium', 'diuretics']
        }

    def check_interactions(self, current_meds, new_meds):
        interactions = []
        for med in new_meds:
            if med in self.interaction_db:
                for existing_med in current_meds:
                    if existing_med in self.interaction_db[med]:
                        interactions.append(f"{med} + {existing_med}")
        return interactions

# ========================= Multimodal Processing Components =========================

class VisionAnalyzer:
    """Vision Transformer for medical image analysis"""

    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        try:
            # Load Vision Transformer
            self.processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
            self.model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224').to(self.device)
            self.model.eval()
            print("✅ Vision Transformer loaded successfully")
        except Exception as e:
            print(f"⚠️ Could not load Vision Transformer: {e}")
            self.processor = None
            self.model = None

    def analyze_image(self, image_path: str) -> Dict:
        """Analyze medical image and return findings"""
        if not self.model:
            return {"error": "Vision model not available"}

        try:
            image = Image.open(image_path).convert('RGB')
            inputs = self.processor(images=image, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)
                logits = outputs.logits
                predicted_class_idx = logits.argmax(-1).item()
                confidence = torch.nn.functional.softmax(logits, dim=-1)[0][predicted_class_idx].item()

            # Medical image analysis results
            analysis = {
                "image_type": "medical_image",
                "findings": [],
                "confidence": confidence,
                "recommendations": []
            }

            # Analyze for common medical conditions in images
            analysis["findings"] = self._interpret_medical_image(image, predicted_class_idx)

            return analysis

        except Exception as e:
            return {"error": f"Image analysis failed: {str(e)}"}

    def _interpret_medical_image(self, image: Image, class_idx: int) -> List[str]:
        """Interpret medical findings from image"""
        findings = []

        # Basic image analysis (would be more sophisticated in production)
        img_array = np.array(image)

        # Check for redness (potential inflammation)
        red_channel = img_array[:,:,0]
        if np.mean(red_channel) > 150:
            findings.append("พบการอักเสบหรือรอยแดง")

        # Check for dark spots
        gray = np.mean(img_array, axis=2)
        if np.std(gray) > 50:
            findings.append("พบจุดหรือรอยที่ผิดปกติ")

        return findings if findings else ["ไม่พบความผิดปกติที่ชัดเจน"]

class AudioAnalyzer:
    """Whisper model for voice/audio analysis"""

    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        try:
            # Load smaller Whisper model for efficiency
            self.processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
            self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny").to(self.device)
            self.model.eval()
            print("✅ Whisper model loaded successfully")
        except Exception as e:
            print(f"⚠️ Could not load Whisper model: {e}")
            self.processor = None
            self.model = None

    def analyze_audio(self, audio_path: str) -> Dict:
        """Analyze audio for medical symptoms"""
        if not self.model:
            return {"error": "Audio model not available"}

        try:
            # Load audio
            audio, sample_rate = librosa.load(audio_path, sr=16000)

            # Transcribe audio
            inputs = self.processor(audio, sampling_rate=sample_rate, return_tensors="pt").to(self.device)

            with torch.no_grad():
                predicted_ids = self.model.generate(inputs.input_features)
                transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

            # Analyze audio characteristics
            audio_features = self._analyze_audio_features(audio, sample_rate)

            return {
                "transcription": transcription,
                "audio_features": audio_features,
                "medical_indicators": self._detect_medical_indicators(audio_features)
            }

        except Exception as e:
            return {"error": f"Audio analysis failed: {str(e)}"}

    def _analyze_audio_features(self, audio: np.ndarray, sample_rate: int) -> Dict:
        """Extract audio features for medical analysis"""
        features = {}

        # Analyze cough patterns
        features["energy"] = np.mean(np.abs(audio))
        features["zero_crossing_rate"] = np.mean(librosa.feature.zero_crossing_rate(audio))

        # Spectral features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
        features["spectral_centroid"] = np.mean(spectral_centroids)

        # MFCC for voice quality
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        features["mfcc_mean"] = np.mean(mfccs, axis=1).tolist()

        return features

    def _detect_medical_indicators(self, audio_features: Dict) -> List[str]:
        """Detect medical indicators from audio features"""
        indicators = []

        # Check for cough patterns
        if audio_features.get("zero_crossing_rate", 0) > 0.1:
            indicators.append("อาจมีอาการไอ")

        # Check for wheezing (high frequency)
        if audio_features.get("spectral_centroid", 0) > 3000:
            indicators.append("อาจมีเสียงหวีด")

        # Check for voice hoarseness
        if audio_features.get("energy", 0) < 0.01:
            indicators.append("เสียงแหบหรืออ่อนแรง")

        return indicators

# ========================= Clarification System =========================

class ClarificationSystem:
    """System for asking follow-up questions"""

    def __init__(self):
        self.question_templates = self._init_question_templates()
        self.asked_questions = set()

    def _init_question_templates(self) -> Dict[str, List[ClarificationQuestion]]:
        """Initialize clarification question templates"""
        return {
            "pain": [
                ClarificationQuestion(
                    question_id="pain_level",
                    category="severity",
                    question_text="ระดับความเจ็บปวดเป็นอย่างไร (1-10)?",
                    options=["1-3 (เล็กน้อย)", "4-6 (ปานกลาง)", "7-9 (รุนแรง)", "10 (รุนแรงมาก)"],
                    required=True
                ),
                ClarificationQuestion(
                    question_id="pain_duration",
                    category="duration",
                    question_text="อาการปวดเริ่มมานานเท่าไร?",
                    options=["น้อยกว่า 1 ชั่วโมง", "1-6 ชั่วโมง", "6-24 ชั่วโมง", "1-3 วัน", "มากกว่า 3 วัน"],
                    required=True
                ),
                ClarificationQuestion(
                    question_id="pain_type",
                    category="characteristics",
                    question_text="ลักษณะการปวดเป็นแบบไหน?",
                    options=["ปวดตุ้บๆ", "ปวดแสบร้อน", "ปวดเสียว", "ปวดตื้อๆ", "ปวดบีบรัด"],
                    required=False
                )
            ],
            "fever": [
                ClarificationQuestion(
                    question_id="fever_temp",
                    category="measurement",
                    question_text="อุณหภูมิร่างกายเท่าไร (ถ้าวัดแล้ว)?",
                    options=["37-37.5°C", "37.5-38°C", "38-39°C", "39-40°C", "มากกว่า 40°C", "ไม่ได้วัด"],
                    required=True
                ),
                ClarificationQuestion(
                    question_id="fever_pattern",
                    category="pattern",
                    question_text="ไข้มีลักษณะอย่างไร?",
                    options=["ไข้ตลอดเวลา", "ไข้ขึ้นๆลงๆ", "ไข้เฉพาะตอนเย็น", "ไข้หนาวสั่น"],
                    required=False
                )
            ],
            "respiratory": [
                ClarificationQuestion(
                    question_id="breathing_difficulty",
                    category="severity",
                    question_text="หายใจลำบากแค่ไหน?",
                    options=["หายใจปกติ", "เหนื่อยเวลาออกแรง", "เหนื่อยเวลาเดิน", "เหนื่อยตลอดเวลา", "หายใจไม่ออก"],
                    required=True,
                    follow_up_questions=["cough_type", "sputum_color"]
                )
            ]
        }

    def generate_clarification(self, symptoms: List[str], context: Dict) -> Optional[ClarificationQuestion]:
        """Generate appropriate clarification question based on symptoms"""
        for symptom in symptoms:
            # Find relevant questions for the symptom
            symptom_key = self._map_symptom_to_category(symptom)
            if symptom_key in self.question_templates:
                questions = self.question_templates[symptom_key]

                # Find unasked required questions
                for question in questions:
                    if question.question_id not in self.asked_questions:
                        if question.required or self._should_ask_optional(question, context):
                            self.asked_questions.add(question.question_id)
                            return question

        return None

    def _map_symptom_to_category(self, symptom: str) -> str:
        """Map symptom to question category"""
        mappings = {
            "pain": ["ปวด", "เจ็บ", "pain", "ache"],
            "fever": ["ไข้", "ร้อน", "fever", "hot"],
            "respiratory": ["หายใจ", "ไอ", "หอบ", "breath", "cough"]
        }

        for category, keywords in mappings.items():
            if any(keyword in symptom.lower() for keyword in keywords):
                return category

        return "general"

    def _should_ask_optional(self, question: ClarificationQuestion, context: Dict) -> bool:
        """Determine if optional question should be asked"""
        # Ask optional questions based on severity or other factors
        severity = context.get("severity", "unknown")
        if severity in ["severe", "รุนแรง"]:
            return True
        return False

    def process_answer(self, question_id: str, answer: str) -> Dict:
        """Process user's answer to clarification question"""
        return {
            "question_id": question_id,
            "answer": answer,
            "timestamp": datetime.now().isoformat(),
            "processed": True
        }

# ========================= Feedback and Learning System =========================

class FeedbackLearningSystem:
    """System for collecting feedback and automated labeling"""

    def __init__(self):
        self.feedback_store = []
        self.labeled_data = []
        self.model_performance = {
            "accuracy": 0.0,
            "helpful_rate": 0.0,
            "total_interactions": 0
        }

    def collect_feedback(self, interaction_id: str, user_id: str, feedback_data: Dict) -> InteractionFeedback:
        """Collect user feedback"""
        feedback = InteractionFeedback(
            interaction_id=interaction_id,
            user_id=user_id,
            timestamp=datetime.now(),
            rating=feedback_data.get("rating", 3),
            helpful=feedback_data.get("helpful", False),
            accurate=feedback_data.get("accurate", False),
            comments=feedback_data.get("comments", ""),
            symptoms_resolved=feedback_data.get("symptoms_resolved", False),
            follow_up_needed=feedback_data.get("follow_up_needed", False)
        )

        self.feedback_store.append(feedback)
        self._update_performance_metrics(feedback)

        return feedback

    def _update_performance_metrics(self, feedback: InteractionFeedback):
        """Update model performance metrics"""
        self.model_performance["total_interactions"] += 1

        # Calculate running averages
        n = self.model_performance["total_interactions"]

        # Update accuracy
        if feedback.accurate:
            self.model_performance["accuracy"] = (
                (self.model_performance["accuracy"] * (n - 1) + 1) / n
            )
        else:
            self.model_performance["accuracy"] = (
                (self.model_performance["accuracy"] * (n - 1)) / n
            )

        # Update helpful rate
        if feedback.helpful:
            self.model_performance["helpful_rate"] = (
                (self.model_performance["helpful_rate"] * (n - 1) + 1) / n
            )
        else:
            self.model_performance["helpful_rate"] = (
                (self.model_performance["helpful_rate"] * (n - 1)) / n
            )

    def auto_label_interaction(self, interaction_data: Dict) -> Dict:
        """Automatically label interaction data for training"""
        labels = {
            "interaction_id": interaction_data.get("id"),
            "timestamp": datetime.now().isoformat(),
            "auto_labels": []
        }

        # Extract symptoms mentioned
        text = interaction_data.get("text", "").lower()

        # Symptom detection
        symptom_keywords = {
            "pain": ["ปวด", "เจ็บ", "pain", "ache"],
            "fever": ["ไข้", "ร้อน", "fever"],
            "cough": ["ไอ", "cough"],
            "fatigue": ["เหนื่อย", "อ่อนเพลีย", "tired"]
        }

        for symptom, keywords in symptom_keywords.items():
            if any(keyword in text for keyword in keywords):
                labels["auto_labels"].append({
                    "type": "symptom",
                    "value": symptom,
                    "confidence": 0.8
                })

        # Urgency detection
        if any(word in text for word in ["ฉุกเฉิน", "เร่งด่วน", "รุนแรง", "emergency"]):
            labels["auto_labels"].append({
                "type": "urgency",
                "value": "high",
                "confidence": 0.9
            })

        # Domain classification
        if "ยา" in text or "medication" in text:
            labels["auto_labels"].append({
                "type": "domain",
                "value": "medication",
                "confidence": 0.85
            })

        self.labeled_data.append(labels)
        return labels

    def generate_training_data(self) -> List[Dict]:
        """Generate training data from feedback and auto-labels"""
        training_data = []

        for feedback in self.feedback_store:
            if feedback.rating >= 4 and feedback.accurate:
                # Use highly-rated accurate interactions for training
                interaction_labels = next(
                    (item for item in self.labeled_data
                     if item["interaction_id"] == feedback.interaction_id),
                    None
                )

                if interaction_labels:
                    training_data.append({
                        "interaction_id": feedback.interaction_id,
                        "labels": interaction_labels["auto_labels"],
                        "feedback": {
                            "rating": feedback.rating,
                            "helpful": feedback.helpful,
                            "accurate": feedback.accurate
                        },
                        "quality_score": feedback.rating / 5.0
                    })

        return training_data

    def get_performance_report(self) -> Dict:
        """Generate performance report"""
        return {
            "metrics": self.model_performance,
            "total_feedback": len(self.feedback_store),
            "labeled_interactions": len(self.labeled_data),
            "average_rating": sum(f.rating for f in self.feedback_store) / len(self.feedback_store) if self.feedback_store else 0,
            "accuracy_rate": self.model_performance["accuracy"],
            "helpful_rate": self.model_performance["helpful_rate"]
        }

# ========================= Enhanced Medical Knowledge Base =========================

class EnhancedMedicalKnowledgeBase:
    """Enhanced medical knowledge with Thai and English support"""

    def __init__(self):
        self.emergency_symptoms = {
            'critical': [
                'หายใจไม่ออก', 'เจ็บหน้าอก', 'ชัก', 'หมดสติ',
                'เลือดออกมาก', 'breathing difficulty', 'chest pain',
                'seizure', 'unconscious', 'severe bleeding'
            ],
            'high': [
                'ไข้สูงมาก', 'ปวดท้องรุนแรง', 'อาเจียนเป็นเลือด',
                'high fever', 'severe abdominal pain', 'vomiting blood'
            ]
        }

        self.symptom_disease_mapping = {
            'ไข้': ['ไข้หวัด', 'ไข้เลือดออก', 'โควิด-19', 'ไข้หวัดใหญ่'],
            'ปวดหัว': ['ไมเกรน', 'ความดันโลหิตสูง', 'เครียด'],
            'ไอ': ['หวัด', 'ภูมิแพ้', 'หอบหืด', 'ปอดอักเสบ'],
            'fever': ['flu', 'dengue', 'COVID-19', 'influenza'],
            'headache': ['migraine', 'hypertension', 'stress'],
            'cough': ['cold', 'allergy', 'asthma', 'pneumonia']
        }

        self.medication_database = {
            'paracetamol': {
                'th_name': 'พาราเซตามอล',
                'uses': ['ลดไข้', 'แก้ปวด'],
                'dosage': '500-1000 mg ทุก 4-6 ชั่วโมง',
                'warnings': ['ไม่เกิน 4000 mg ต่อวัน', 'ระวังในผู้ป่วยโรคตับ']
            },
            'aspirin': {
                'th_name': 'แอสไพริน',
                'uses': ['แก้ปวด', 'ลดไข้', 'ป้องกันลิ่มเลือด'],
                'dosage': '325-650 mg ทุก 4 ชั่วโมง',
                'warnings': ['ไม่ใช้ในเด็กต่ำกว่า 18 ปี', 'ระวังในผู้ป่วยโรคกระเพาะ']
            }
        }

        self.first_aid_procedures = {
            'cpr': {
                'th_name': 'การช่วยฟื้นคืนชีพ',
                'steps': [
                    'ตรวจสอบการตอบสนอง',
                    'โทร 1669',
                    'กดหน้าอก 30 ครั้ง',
                    'เป่าปาก 2 ครั้ง',
                    'ทำซ้ำจนกว่าความช่วยเหลือจะมาถึง'
                ]
            },
            'bleeding': {
                'th_name': 'การห้ามเลือด',
                'steps': [
                    'กดแผลด้วยผ้าสะอาด',
                    'ยกส่วนที่บาดเจ็บให้สูง',
                    'กดต่อเนื่อง 10-15 นาที',
                    'ถ้าเลือดไม่หยุด โทร 1669'
                ]
            }
        }

        self.health_tips = {
            'prevention': [
                'ล้างมือบ่อยๆ ด้วยสบู่',
                'สวมหน้ากากในที่แออัด',
                'ออกกำลังกายสม่ำเสมอ',
                'นอนหลับพักผ่อนให้เพียงพอ',
                'ดื่มน้ำให้เพียงพอ'
            ],
            'nutrition': [
                'กินผักผลไม้ 5 หมู่',
                'ลดอาหารหวาน มัน เค็ม',
                'เลือกโปรตีนไขมันต่ำ',
                'กินอาหารให้ครบ 5 หมู่'
            ]
        }

    def check_emergency_level(self, symptoms: List[str]) -> UrgencyLevel:
        """Check urgency level from symptoms"""
        symptoms_lower = [s.lower() for s in symptoms]

        for symptom in symptoms_lower:
            if any(critical in symptom for critical in self.emergency_symptoms['critical']):
                return UrgencyLevel.CRITICAL
            elif any(high in symptom for high in self.emergency_symptoms['high']):
                return UrgencyLevel.HIGH

        return UrgencyLevel.MEDIUM

    def get_possible_conditions(self, symptoms: List[str]) -> List[str]:
        """Get possible conditions based on symptoms"""
        conditions = []
        for symptom in symptoms:
            symptom_lower = symptom.lower()
            for key, diseases in self.symptom_disease_mapping.items():
                if key in symptom_lower:
                    conditions.extend(diseases)

        return list(set(conditions))

    def get_medication_info(self, medication_name: str) -> Dict:
        """Get medication information"""
        med_name = medication_name.lower()

        for key, info in self.medication_database.items():
            if key in med_name or info['th_name'] in medication_name:
                return info

        return {
            "error": f"ไม่พบข้อมูลยา {medication_name}",
            "suggestion": "กรุณาปรึกษาเภสัชกรหรือแพทย์"
        }

    def get_first_aid_steps(self, emergency_type: str) -> List[str]:
        """Get first aid steps for emergency"""
        emergency_lower = emergency_type.lower()

        for key, info in self.first_aid_procedures.items():
            if key in emergency_lower or any(term in emergency_lower for term in [info['th_name'], key]):
                return info['steps']

        return ["โทร 1669 เพื่อขอความช่วยเหลือทันที"]

    def get_health_recommendations(self, category: str = "prevention") -> List[str]:
        """Get health recommendations"""
        return self.health_tips.get(category, self.health_tips['prevention'])

# ========================= Enhanced Thai Medical AI Agent =========================

class EnhancedThaiMedicalAI:
    """Main AI Agent class with multimodal capabilities"""

    def __init__(self):
        print("🚀 Initializing Enhanced Thai Medical AI Agent...")

        # Initialize core components
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"📱 Device: {self.device}")

        # Initialize storage and compliance
        self.profile_storage = UserProfileStorage()
        self.pdpa = PDPACompliance()

        # Initialize AI components
        self.knowledge_base = EnhancedMedicalKnowledgeBase()
        self.triage_system = EmergencyTriageSystem()
        self.drug_checker = DrugInteractionChecker()
        self.clarification_system = ClarificationSystem()
        self.feedback_system = FeedbackLearningSystem()

        # Initialize multimodal components
        self.vision_analyzer = VisionAnalyzer()
        self.audio_analyzer = AudioAnalyzer()

        # Load main language model
        self._load_thai_llm()

        # Initialize conversation state
        self.conversation_history = []
        self.current_user_id = None
        self.current_session = {}

        print("✅ Enhanced Thai Medical AI Agent initialized successfully!")

    def _load_thai_llm(self):

        """Load typhoon2.1-gemma3-4b model with PEFT adapter"""
        try:
            base_model_name = "scb10x/typhoon2.1-gemma3-4b"
            adapter_path = "/content/drive/MyDrive/V89Technology/typhoon21-gemma3-4b-medCare-finetuned/checkpoint-120"

            # Configure for T4 GPU efficiency
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                llm_int8_threshold=6.0,
            )

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Load base model
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
               quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.bfloat16,
                trust_remote_code=True,
                low_cpu_mem_usage=True,
                #use_cache=False,
            )

            # Load PEFT adapter
            from peft import PeftModel
            self.model = PeftModel.from_pretrained(
                self.model,
                adapter_path,
                adapter_name="medical_adapter"
            )

            # Set model to evaluation mode
            self.model.eval()

            print("✅ typhoon2.1-gemma3-4b model with medical PEFT adapter loaded successfully")

        except Exception as e:
            print(f"⚠️ Error loading typhoon2.1-gemma3-4b LLM: {e}")
            # Fallback to pipeline
            try:
                self.pipeline = pipeline(
                    "text-generation",
                    model="scb10x/typhoon2.1-gemma3-4b",
                    device=0 if torch.cuda.is_available() else -1
		 #use_cache=False
                )
                self.model = None
                self.tokenizer = None
                print("✅ Fallback model loaded")
            except:
                self.model = None
                self.tokenizer = None
                self.pipeline = None
                print("❌ No language model available")


    def create_user_session(self, user_data: Dict) -> str:
        """Create new user session with consent management"""
        # Request basic consent
        consent_request = self.pdpa.request_consent(
            user_data.get('temp_id', str(uuid.uuid4())),
            ConsentType.DATA_COLLECTION
        )

        return consent_request['consent_id']

    def process_consent(self, consent_id: str, granted: bool, user_data: Dict) -> Dict:
        """Process user consent and create profile if granted"""
        if granted:
            # Create user profile
            profile = self.profile_storage.create_user_profile(user_data)
            self.current_user_id = profile.user_id

            # Record consent
            user_consent = UserConsent(
                user_id=profile.user_id,
                consent_type=ConsentType.DATA_COLLECTION,
                granted=True,
                timestamp=datetime.now()
            )
            self.pdpa.record_consent(user_consent)

            return {
                "success": True,
                "user_id": profile.user_id,
                "message": "ยินดีต้อนรับ! ข้อมูลของคุณจะถูกเก็บรักษาอย่างปลอดภัย"
            }
        else:
            return {
                "success": False,
                "message": "คุณสามารถใช้บริการโดยไม่จัดเก็บข้อมูลส่วนบุคคล แต่การให้คำแนะนำอาจมีประสิทธิภาพจำกัด"
            }

    def process_multimodal_input(self, text: str = "", image_path: str = "", audio_path: str = "") -> Dict:
        """Process multimodal input with Chain of Thought reasoning"""

        # Chain of Thought reasoning steps
        cot_analysis = {
            "step_1_input_analysis": {},
            "step_2_symptom_extraction": {},
            "step_3_multimodal_fusion": {},
            "step_4_medical_reasoning": {},
            "step_5_recommendation": {}
        }

        # Step 1: Analyze each input modality
        if text:
            cot_analysis["step_1_input_analysis"]["text"] = {
                "content": text,
                "language": "thai" if any('\u0e00' <= char <= '\u0e7f' for char in text) else "english",
                "length": len(text.split()),
                "urgency_keywords": [word for word in text.split() if word in ["เร่งด่วน", "ฉุกเฉิน", "emergency"]]
            }

        if image_path:
            image_analysis = self.vision_analyzer.analyze_image(image_path)
            cot_analysis["step_1_input_analysis"]["image"] = image_analysis

        if audio_path:
            audio_analysis = self.audio_analyzer.analyze_audio(audio_path)
            cot_analysis["step_1_input_analysis"]["audio"] = audio_analysis

        # Step 2: Extract symptoms from all modalities
        symptoms = []
        if text:
            text_symptoms = self._extract_symptoms_from_text(text)
            symptoms.extend(text_symptoms)
            cot_analysis["step_2_symptom_extraction"]["text_symptoms"] = text_symptoms

        if image_path and "findings" in cot_analysis["step_1_input_analysis"].get("image", {}):
            image_symptoms = cot_analysis["step_1_input_analysis"]["image"]["findings"]
            symptoms.extend(image_symptoms)
            cot_analysis["step_2_symptom_extraction"]["image_symptoms"] = image_symptoms

        if audio_path and "medical_indicators" in cot_analysis["step_1_input_analysis"].get("audio", {}):
            audio_symptoms = cot_analysis["step_1_input_analysis"]["audio"]["medical_indicators"]
            symptoms.extend(audio_symptoms)
            cot_analysis["step_2_symptom_extraction"]["audio_symptoms"] = audio_symptoms

        # Step 3: Fuse multimodal information
        confidence_scores = {}
        if text and image_path:
            # Cross-validate symptoms between text and image
            text_set = set(cot_analysis["step_2_symptom_extraction"].get("text_symptoms", []))
            image_set = set(cot_analysis["step_2_symptom_extraction"].get("image_symptoms", []))
            overlap = text_set.intersection(image_set)
            if overlap:
                confidence_scores["cross_modal_validation"] = len(overlap) / len(text_set.union(image_set))

        cot_analysis["step_3_multimodal_fusion"] = {
            "total_symptoms": symptoms,
            "confidence_scores": confidence_scores,
            "modalities_used": [k for k in ["text", "image", "audio"] if k in cot_analysis["step_1_input_analysis"]]
        }

        # Step 4: Medical reasoning
        urgency = self.triage_system.assess_urgency(text, cot_analysis["step_1_input_analysis"].get("image"))
        possible_conditions = self.knowledge_base.get_possible_conditions(symptoms)
        emergency_level = self.knowledge_base.check_emergency_level(symptoms)

        cot_analysis["step_4_medical_reasoning"] = {
            "urgency_level": urgency,
            "possible_conditions": possible_conditions,
            "emergency_assessment": emergency_level.value,
            "requires_immediate_attention": emergency_level in [UrgencyLevel.CRITICAL, UrgencyLevel.HIGH]
        }

        # Step 5: Generate recommendations
        recommendations = self._generate_recommendations(cot_analysis)
        cot_analysis["step_5_recommendation"] = recommendations

        return cot_analysis

    def _extract_symptoms_from_text(self, text: str) -> List[str]:
        """Extract symptoms from text using Thai NLP"""
        symptoms = []
        text_lower = text.lower()

        # Define symptom patterns
        symptom_patterns = {
            'ไข้': ['ไข้', 'ร้อน', 'fever', 'temperature'],
            'ปวดหัว': ['ปวดหัว', 'headache', 'หัวเจ็บ'],
            'ไอ': ['ไอ', 'cough', 'ho'],
            'เจ็บคอ': ['เจ็บคอ', 'คอแห้ง', 'sore throat'],
            'วิงเวียน': ['เวียนหัว', 'วิงเวียน', 'dizzy'],
            'เหนื่อย': ['เหนื่อย', 'อ่อนเพลีย', 'tired', 'fatigue'],
            'คลื่นไส้': ['คลื่นไส้', 'อยากอาเจียน', 'nausea'],
            'ปวดท้อง': ['ปวดท้อง', 'ท้องเจ็บ', 'abdominal pain'],
            'หายใจลำบาก': ['หายใจลำบาก', 'หอบเหนื่อย', 'breathing difficulty']
        }

        for symptom, patterns in symptom_patterns.items():
            if any(pattern in text_lower for pattern in patterns):
                symptoms.append(symptom)

        return symptoms

    def _generate_recommendations(self, cot_analysis: Dict) -> Dict:
        """Generate medical recommendations based on CoT analysis"""
        recommendations = {
            "immediate_actions": [],
            "medical_advice": [],
            "lifestyle_recommendations": [],
            "follow_up": [],
            "emergency_warning": None
        }

        emergency_level = cot_analysis["step_4_medical_reasoning"]["emergency_assessment"]

        # Emergency recommendations
        if emergency_level == "critical":
            recommendations["immediate_actions"] = [
                "🚨 เรียกรถพยาบาลทันที โทร 1669",
                "🏥 ไปโรงพยาบาลฉุกเฉินทันที",
                "📞 แจ้งญาติใกล้ชิด"
            ]
            recommendations["emergency_warning"] = "สถานการณ์ฉุกเฉิน - ต้องได้รับการรักษาทันที"

        elif emergency_level == "high":
            recommendations["immediate_actions"] = [
                "🏥 ไปพบแพทย์ในวันนี้",
                "📋 เตรียมรายการอาการและยาที่กิน",
                "📞 โทรสอบถามโรงพยาบาลก่อนไป"
            ]

        # General medical advice based on symptoms
        symptoms = cot_analysis["step_3_multimodal_fusion"]["total_symptoms"]

        if "ไข้" in symptoms:
            recommendations["medical_advice"].extend([
                "💊 พิจารณาทานยาลดไข้ตามเหมาะสม",
                "🌡️ วัดอุณหภูมิร่างกายทุก 4 ชั่วโมง",
                "💧 ดื่มน้ำเปล่าให้เพียงพอ"
            ])

        if "ไอ" in symptoms:
            recommendations["medical_advice"].extend([
                "🍯 ดื่มน้ำอุ่นผสมน้ำผึ้ง",
                "🚭 หลีกเลี่ยงควันและฝุ่น",
                "😷 สวมหน้ากากอนามัย"
            ])

        if "ปวดหัว" in symptoms:
            recommendations["medical_advice"].extend([
                "😴 พักผ่อนในที่เงียบและมืด",
                "💆 ประคบเย็นที่หน้าผาก",
                "💊 ทานยาแก้ปวดหัวตามต้อง"
            ])

        # Lifestyle recommendations
        recommendations["lifestyle_recommendations"] = [
            "🥗 รับประทานอาหารที่มีประโยชน์",
            "💤 นอนหลับให้เพียงพอ 7-9 ชั่วโมง",
            "🚶‍♂️ ออกกำลังกายเบาๆ หากรู้สึกดีขึ้น",
            "🧘‍♀️ จัดการความเครียด"
        ]

        # Follow-up recommendations
        if emergency_level in ["medium", "low"]:
            recommendations["follow_up"] = [
                "📅 นัดติดตามอาการใน 2-3 วัน",
                "📝 บันทึกอาการและการเปลี่ยนแปลง",
                "🏥 ไปพบแพทย์หากอาการไม่ดีขึ้น"
            ]

        return recommendations

    def generate_response(self, input_text: str, context: Dict = None) -> str:
        """Generate response using typhoon2.1-gemma3-4b with medical context"""
        if not self.model and not self.pipeline:
            return self._generate_rule_based_response(input_text, context)

        # Prepare medical context
        medical_context = ""
        if context:
            if "symptoms" in context:
                medical_context += f"อาการที่พบ: {', '.join(context['symptoms'])}\n"
            if "urgency" in context:
                medical_context += f"ระดับความเร่งด่วน: {context['urgency']}\n"
            if "recommendations" in context:
                medical_context += f"คำแนะนำ: {context['recommendations']}\n"

        # Create prompt for Thai medical assistant (typhoon2.1-gemma3-4b format)
        messages = [
            {"role": "system", "content": "คุณเป็นผู้ช่วยแพทย์ AI ที่เชี่ยวชาญด้านการแพทย์ กรุณาให้คำแนะนำทางการแพทย์ที่เหมาะสม"},
            {"role": "user", "content": f"{medical_context}\n\n{input_text}"}
        ]

        # Format for typhoon2.1-gemma3-4b
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        try:
            if self.model and self.tokenizer:
                inputs = self.tokenizer(text, return_tensors="pt", max_length=2048, truncation=True)
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=512,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=self.tokenizer.eos_token_id,
                        repetition_penalty=1.1,
                        top_p=0.9,
		     use_cache=True
                    )

                response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                # Extract only the assistant's response
                response = response.split("<|im_start|>assistant\n")[-1].strip()
                response = response.split("<|im_end|>")[0].strip()

            else:
                # Use pipeline fallback
                result = self.pipeline(text, max_length=200, temperature=0.7)
                response = result[0]['generated_text'].split("assistant\n")[-1].strip()

            return response if response else self._generate_rule_based_response(input_text, context)

        except Exception as e:
            print(f"Error in text generation: {e}")
            return self._generate_rule_based_response(input_text, context)

    def _generate_rule_based_response(self, input_text: str, context: Dict = None) -> str:
        """Generate rule-based response as fallback"""
        responses = [
            "ขอบคุณที่ให้ข้อมูล ฉันจะช่วยวิเคราะห์อาการและให้คำแนะนำที่เหมาะสม",
            "จากอาการที่คุณแจ้งมา ฉันขอแนะนำให้ปฏิบัติตามคำแนะนำที่ให้ไป",
            "หากอาการไม่ดีขึ้นหรือรุนแรงขึ้น กรุณาปรึกษาแพทย์ทันที"
        ]

        if context and context.get("emergency_level") == "critical":
            return "🚨 จากอาการที่แจ้งมาอาจเป็นสถานการณ์ฉุกเฉิน กรุณาไปโรงพยาบาลหรือโทร 1669 ทันที"

        return responses[0]

    def ask_clarification(self, symptoms: List[str]) -> Optional[ClarificationQuestion]:
        """Ask clarification questions"""
        context = {
            "severity": "medium",
            "duration": "unknown"
        }
        return self.clarification_system.generate_clarification(symptoms, context)

    def process_feedback(self, interaction_id: str, feedback_data: Dict) -> Dict:
        """Process user feedback"""
        if not self.current_user_id:
            return {"error": "ไม่พบข้อมูลผู้ใช้"}

        feedback = self.feedback_system.collect_feedback(
            interaction_id, self.current_user_id, feedback_data
        )

        # Store in database
        self.profile_storage.store_feedback(feedback)

        return {
            "success": True,
            "message": "ขอบคุณสำหรับข้อเสนอแนะ จะช่วยปรับปรุงการให้บริการ"
        }

    def get_conversation_summary(self) -> Dict:
        """Get conversation summary and analytics"""
        return {
            "total_interactions": len(self.conversation_history),
            "current_user": self.current_user_id,
            "performance_metrics": self.feedback_system.get_performance_report(),
            "session_duration": datetime.now().isoformat()
        }

# ========================= Gradio Interface =========================

def create_gradio_interface():
    """Create Gradio interface for the medical AI"""

    # Initialize the AI agent
    medical_ai = EnhancedThaiMedicalAI()

    def process_chat(message, history, image, audio):
        """Process chat with multimodal input"""
        try:
            # Handle file uploads
            image_path = image.name if image else ""
            audio_path = audio.name if audio else ""

            # Process multimodal input with CoT
            cot_result = medical_ai.process_multimodal_input(
                text=message,
                image_path=image_path,
                audio_path=audio_path
            )

            # Extract recommendations
            recommendations = cot_result["step_5_recommendation"]

            # Generate AI response
            context = {
                "symptoms": cot_result["step_3_multimodal_fusion"]["total_symptoms"],
                "urgency": cot_result["step_4_medical_reasoning"]["urgency_level"],
                "emergency_level": cot_result["step_4_medical_reasoning"]["emergency_assessment"],
                "recommendations": recommendations
            }

            ai_response = medical_ai.generate_response(message, context)

            # Format response
            response_parts = [ai_response]

            # Add emergency warning if needed
            if recommendations.get("emergency_warning"):
                response_parts.insert(0, f"⚠️ {recommendations['emergency_warning']}")

            # Add immediate actions
            if recommendations["immediate_actions"]:
                response_parts.append("\n🚨 การดำเนินการทันที:")
                response_parts.extend([f"• {action}" for action in recommendations["immediate_actions"]])

            # Add medical advice
            if recommendations["medical_advice"]:
                response_parts.append("\n💡 คำแนะนำทางการแพทย์:")
                response_parts.extend([f"• {advice}" for advice in recommendations["medical_advice"]])

            # Add follow-up
            if recommendations["follow_up"]:
                response_parts.append("\n📋 การติดตาม:")
                response_parts.extend([f"• {follow}" for follow in recommendations["follow_up"]])

            final_response = "\n".join(response_parts)

            # Update conversation history
            history.append([message, final_response])

            return "", history, None, None

        except Exception as e:
            error_msg = f"เกิดข้อผิดพลาด: {str(e)}\nกรุณาลองใหม่อีกครั้ง หรือปรึกษาแพทย์โดยตรง"
            history.append([message, error_msg])
            return "", history, None, None

    def submit_feedback(rating, helpful, accurate, comments):
        """Submit user feedback"""
        try:
            feedback_data = {
                "rating": rating,
                "helpful": helpful == "Yes",
                "accurate": accurate == "Yes",
                "comments": comments
            }

            # For demo, use a dummy interaction ID
            interaction_id = str(uuid.uuid4())
            result = medical_ai.process_feedback(interaction_id, feedback_data)

            return "ขอบคุณสำหรับข้อเสนอแนะ!"

        except Exception as e:
            return f"เกิดข้อผิดพลาดในการส่งข้อเสนอแนะ: {str(e)}"

    def create_user_profile(name, age, gender, medical_history, medications, allergies):
        """Create user profile"""
        try:
            user_data = {
                "name": name,
                "age": int(age) if age else 0,
                "gender": gender,
                "medical_history": medical_history.split(",") if medical_history else [],
                "medications": medications.split(",") if medications else [],
                "allergies": allergies.split(",") if allergies else []
            }

            consent_id = medical_ai.create_user_session(user_data)

            # For demo, automatically grant consent
            result = medical_ai.process_consent(consent_id, True, user_data)

            if result["success"]:
                return f"สร้างโปรไฟล์สำเร็จ! User ID: {result['user_id'][:8]}..."
            else:
                return result["message"]

        except Exception as e:
            return f"เกิดข้อผิดพลาด: {str(e)}"

    # Create Gradio interface
    with gr.Blocks(
        title="Enhanced Thai Medical Care AI Agent",
        theme=gr.themes.Soft(),
        css="""
        .container { max-width: 1200px; margin: auto; }
        .emergency { background-color: #fee; border-left: 4px solid #f00; padding: 10px; }
        .warning { background-color: #ffeaa7; border-left: 4px solid #f39c12; padding: 10px; }
        """
    ) as demo:

        gr.Markdown(
            """
            # 🏥 ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำภาษาไทย
            ## Enhanced Thai Medical Care AI Agent Chat Multi-Modal with Chain of Thought

            **⚠️ ข้อจำกัดความรับผิดชอบ:**
            - ระบบนี้เป็นเพียงผู้ช่วยให้คำแนะนำเบื้องต้นเท่านั้น
            - ไม่สามารถทดแทนการปรึกษาแพทย์ได้
            - กรุณาปรึกษาแพทย์สำหรับการวินิจฉัยและการรักษาที่แม่นยำ
            """
        )

        with gr.Tabs():
            # Chat Interface
            with gr.TabItem("💬 แชทปรึกษา"):
                with gr.Row():
                    with gr.Column(scale=2):
                        chatbot = gr.Chatbot(
                            height=500,
                            label="การสนทนา",
                            placeholder="เริ่มสนทนาด้วยการพิมพ์อาการหรือคำถามของคุณ..."
                        )

                        with gr.Row():
                            msg = gr.Textbox(
                                label="ข้อความ",
                                placeholder="พิมพ์อาการหรือคำถามของคุณ...",
                                lines=2
                            )

                        with gr.Row():
                            image_input = gr.File(
                                label="อัปโหลดภาพ (เช่น ผื่น, แผล, ฯลฯ)",
                                file_types=["image"]
                            )
                            audio_input = gr.File(
                                label="อัปโหลดเสียง (เช่น เสียงไอ, หายใจ)",
                                file_types=["audio"]
                            )

                        submit_btn = gr.Button("ส่งข้อความ", variant="primary")

                # Process chat
                submit_btn.click(
                    process_chat,
                    inputs=[msg, chatbot, image_input, audio_input],
                    outputs=[msg, chatbot, image_input, audio_input]
                )

                msg.submit(
                    process_chat,
                    inputs=[msg, chatbot, image_input, audio_input],
                    outputs=[msg, chatbot, image_input, audio_input]
                )

            # User Profile
            with gr.TabItem("👤 โปรไฟล์ผู้ใช้"):
                gr.Markdown("### สร้างโปรไฟล์เพื่อรับคำแนะนำที่เหมาะสมกับคุณ")

                with gr.Row():
                    with gr.Column():
                        profile_name = gr.Textbox(label="ชื่อ-นามสกุล")
                        profile_age = gr.Number(label="อายุ", minimum=0, maximum=150)
                        profile_gender = gr.Dropdown(
                            label="เพศ",
                            choices=["ชาย", "หญิง", "ไม่ระบุ"]
                        )

                    with gr.Column():
                        profile_medical_history = gr.Textbox(
                            label="ประวัติการเจ็บป่วย (คั่นด้วยคอมม่า)",
                            placeholder="เช่น เบาหวาน, ความดันโลหิตสูง"
                        )
                        profile_medications = gr.Textbox(
                            label="ยาที่กินอยู่ปัจจุบัน (คั่นด้วยคอมม่า)",
                            placeholder="เช่น metformin, amlodipine"
                        )
                        profile_allergies = gr.Textbox(
                            label="ประวัติการแพ้ยาหรือสารอื่นๆ (คั่นด้วยคอมม่า)",
                            placeholder="เช่น penicillin, กุ้ง, ไข่"
                        )

                create_profile_btn = gr.Button("สร้างโปรไฟล์", variant="primary")
                profile_status = gr.Textbox(label="สถานะ", interactive=False)

                create_profile_btn.click(
                    create_user_profile,
                    inputs=[profile_name, profile_age, profile_gender, profile_medical_history, profile_medications, profile_allergies],
                    outputs=[profile_status]
                )

            # Feedback Interface
            with gr.TabItem("📝 ให้ความคิดเห็น"):
                gr.Markdown("### ช่วยปรับปรุงระบบด้วยการให้ความคิดเห็น")

                with gr.Row():
                    with gr.Column():
                        feedback_rating = gr.Slider(
                            label="คะแนนความพอใจ",
                            minimum=1,
                            maximum=5,
                            step=1,
                            value=3
                        )
                        feedback_helpful = gr.Radio(
                            label="คำแนะนำมีประโยชน์หรือไม่?",
                            choices=["Yes", "No"],
                            value="Yes"
                        )
                        feedback_accurate = gr.Radio(
                            label="ข้อมูลถูกต้องแม่นยำหรือไม่?",
                            choices=["Yes", "No"],
                            value="Yes"
                        )

                    with gr.Column():
                        feedback_comments = gr.Textbox(
                            label="ข้อเสนอแนะเพิ่มเติม",
                            lines=4,
                            placeholder="กรุณาแสดงความคิดเห็นเพื่อช่วยปรับปรุงระบบ..."
                        )

                submit_feedback_btn = gr.Button("ส่งความคิดเห็น", variant="primary")
                feedback_status = gr.Textbox(label="สถานะ", interactive=False)

                submit_feedback_btn.click(
                    submit_feedback,
                    inputs=[feedback_rating, feedback_helpful, feedback_accurate, feedback_comments],
                    outputs=[feedback_status]
                )

            # Emergency Information
            with gr.TabItem("🚨 ข้อมูลฉุกเฉิน"):
                gr.Markdown("""
                ### 📞 เบอร์โทรฉุกเฉิน

                **🚑 ฉุกเฉิน:**
                - **1669** - ศูนย์นเรนทร์ฉุกเฉิน (24 ชั่วโมง)
                - **1646** - ศูนย์ความช่วยเหลือทางการแพทย์

                **🏥 โรงพยาบาล:**
                - **โรงพยาบาลรัฐบาล:** 1668
                - **ศูนย์ข้อมูลยา:** 1556

                **🏥 เครือข่ายโรงพยาบาล:**
                - โรงพยาบาลชลบุรี: 038-274-100
                - โรงพยาบาลพัทยา: 038-429-422
                - โรงพyาบาลสมิติเวช: 038-235-000

                ### 🩺 อาการที่ต้องรีบไปโรงพยาบาล

                **🚨 อาการฉุกเฉินร้ายแรง:**
                - เจ็บหน้าอกรุนแรง
                - หายใจไม่ออก
                - ชัก
                - หมดสติ
                - มีเลือดออกมาก
                - อาเจียนเป็นเลือด
                - ปวดท้องรุนแรงฉับพลัน

                **⚠️ อาการที่ควรพบแพทย์โดยเร็ว:**
                - ไข้สูงมาก (มากกว่า 39°C)
                - ปวดหัวรุนแรงผิดปกติ
                - ผื่นแพร่งเร็ว
                - ไอเป็นเลือด
                - ปัสสาวะเป็นเลือด

                ### 🏠 การปฐมพยาบาลเบื้องต้น

                **💔 CPR:**
                1. ตรวจสอบการตอบสนอง
                2. โทร 1669
                3. กดหน้าอก 30 ครั้ง
                4. เป่าปาก 2 ครั้ง
                5. ทำซ้ำจนความช่วยเหลือมาถึง

                **🩸 การห้ามเลือด:**
                1. กดแผลด้วยผ้าสะอาด
                2. ยกส่วนที่บาดเจ็บให้สูง
                3. กดต่อเนื่อง 10-15 นาที
                4. ถ้าเลือดไม่หยุด โทร 1669

                **🔥 แผลไฟไหม้:**
                1. ล้างด้วยน้ำเย็นทันที
                2. ถอดเสื้อผ้าที่ร้อน
                3. ใช้ผ้าสะอาดคลุม
                4. ไปโรงพยาบาลหากรุนแรง
                """)

            # System Analytics
            with gr.TabItem("📊 สถิติระบบ"):
                gr.Markdown("### 📈 การใช้งานและประสิทธิภาพระบบ")

                def get_system_stats():
                    """Get system statistics"""
                    try:
                        stats = medical_ai.get_conversation_summary()
                        performance = stats["performance_metrics"]

                        return (
                            f"จำนวนการสนทนาทั้งหมด: {stats['total_interactions']}",
                            f"อัตราความแม่นยำ: {performance['accuracy']:.2%}",
                            f"อัตราความมีประโยชน์: {performance['helpful_rate']:.2%}",
                            f"คะแนนเฉลี่ย: {performance.get('average_rating', 0):.1f}/5.0",
                            f"จำนวนข้อเสนอแนะ: {performance['total_feedback']}"
                        )
                    except Exception as e:
                        return (f"เกิดข้อผิดพลาด: {str(e)}",) * 5

                with gr.Row():
                    stats_btn = gr.Button("อัปเดตสถิติ", variant="secondary")

                with gr.Row():
                    with gr.Column():
                        total_interactions = gr.Textbox(label="การสนทนาทั้งหมด", interactive=False)
                        accuracy_rate = gr.Textbox(label="อัตราความแม่นยำ", interactive=False)
                        helpful_rate = gr.Textbox(label="อัตราความมีประโยชน์", interactive=False)
                    with gr.Column():
                        avg_rating = gr.Textbox(label="คะแนนเฉลี่ย", interactive=False)
                        total_feedback = gr.Textbox(label="จำนวนข้อเสนอแนะ", interactive=False)

                stats_btn.click(
                    get_system_stats,
                    outputs=[total_interactions, accuracy_rate, helpful_rate, avg_rating, total_feedback]
                )

            # Help and Documentation
            with gr.TabItem("❓ วิธีใช้งาน"):
                gr.Markdown("""
                ## 📖 คู่มือการใช้งาน ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำภาษาไทย

                ### 🎯 วัตถุประสงค์
                ระบบนี้พัฒนาขึ้นเพื่อให้ความช่วยเหลือเบื้องต้นทางการแพทย์และข้อมูลสุขภาพ โดยใช้เทคโนโลยี AI และ Multi-Modal Processing

                ### 💻 วิธีการใช้งาน

                **1. 💬 แชทปรึกษา:**
                - พิมพ์อาการหรือคำถามเกี่ยวกับสุขภาพ
                - สามารถอัปโหลดรูปภาพ (แผล, ผื่น, ลักษณะผิดปกติ)
                - อัปโหลดไฟล์เสียง (เสียงไอ, หายใจผิดปกติ)
                - ระบบจะวิเคราะห์และให้คำแนะนำ

                **2. 👤 สร้างโปรไฟล์:**
                - กรอกข้อมูลส่วนตัว (ชื่อ, อายุ, เพศ)
                - ประวัติการเจ็บป่วย
                - ยาที่กินอยู่ปัจจุบัน
                - ประวัติการแพ้ยา

                **3. 📝 ให้ความคิดเห็น:**
                - ประเมินความพอใจ (1-5 คะแนน)
                - ระบุว่าคำแนะนำมีประโยชน์หรือไม่
                - ประเมินความแม่นยำของข้อมูล
                - แสดงความคิดเห็นเพิ่มเติม

                ### 🔧 ฟีเจอร์พิเศษ

                **🧠 Chain of Thought (CoT) Processing:**
                - วิเคราะห์อาการแบบขั้นตอน
                - ประมวลผลข้อมูลหลายประเภท (ข้อความ, รูป, เสียง)
                - ให้เหตุผลในการวินิจฉัยเบื้องต้น

                **🔒 PDPA Compliance:**
                - ปกป้องข้อมูลส่วนบุคคลตามกฎหมาย
                - ขอความยินยอมก่อนใช้ข้อมูล
                - เข้ารหัสข้อมูลสำคัญ

                **🚨 Emergency Triage:**
                - ประเมินระดับความร้ายแรง
                - แจ้งเตือนฉุกเฉิน
                - แนะนำการดำเนินการที่เหมาะสม

                ### ⚠️ ข้อจำกัดและการใช้งาน

                **สิ่งที่ระบบทำได้:**
                - ให้ข้อมูลทางการแพทย์เบื้องต้น
                - ประเมินความร้ายแรงของอาการ
                - แนะนำการปฐมพยาบาล
                - ให้คำแนะนำเรื่องสุขภาพทั่วไป

                **สิ่งที่ระบบทำไม่ได้:**
                - วินิจฉัยโรคแทนแพทย์
                - สั่งจ่ายยาหรือรักษาโรค
                - รับประกันความถูกต้อง 100%
                - ทดแทนการพบแพทย์เมื่อจำเป็น

                ### 📞 เมื่อไหร่ควรพบแพทย์

                **ทันที (โทร 1669):**
                - อาการฉุกเฉินร้ายแรง
                - เจ็บหน้าอก, หายใจไม่ออก
                - ชัก, หมดสติ
                - เลือดออกมาก

                **โดยเร็ว (ภายใน 24 ชั่วโมง):**
                - ไข้สูงไม่ลด
                - ปวดรุนแรงต่อเนื่อง
                - อาการแปลกใหม่
                - ไม่แน่ใจเรื่องยา

                ### 🔍 เทคโนโลยีที่ใช้

                **🤖 AI Models:**
                - **typhoon21-gemma3-4b** with Medical Fine-tuning สำหรับการสนทนาภาษาไทย
                - Vision Transformer สำหรับวิเคราะห์รูปภาพ
                - Whisper สำหรับประมวลผลเสียง

                **💾 Data Processing:**
                - Natural Language Processing ภาษาไทย
                - Image Analysis สำหรับอาการทางผิวหนัง
                - Audio Analysis สำหรับอาการทางระบบหายใจ

                **🛡️ Security:**
                - การเข้ารหัสข้อมูล (AES-256)
                - Audit Log สำหรับการตรวจสอบ
                - Anonymization ข้อมูลส่วนบุคคล

                ### 🙋‍♀️ คำถามที่พบบ่อย

                **Q: ระบบนี้ปลอดภัยหรือไม่?**
                A: ใช่ ระบบปฏิบัติตาม PDPA และเข้ารหัสข้อมูลทั้งหมด

                **Q: สามารถใช้แทนการพบแพทย์ได้หรือไม่?**
                A: ไม่ได้ ระบบให้คำแนะนำเบื้องต้นเท่านั้น ควรพบแพทย์สำหรับการวินิจฉัยและรักษาที่แม่นยำ

                **Q: ข้อมูลจะถูกเก็บไว้นานแค่ไหน?**
                A: ข้อมูลจะถูกเก็บตามกฎหมาย PDPA และจะลบออกเมื่อหมดความจำเป็น

                **Q: ระบบรองรับภาษาอื่นหรือไม่?**
                A: ปัจจุบันรองรับภาษาไทยเป็นหลัก แต่สามารถเข้าใจภาษาอังกฤษเบื้องต้นได้

                ### 📧 ติดต่อสอบถาม
                หากมีคำถามหรือข้อสงสัยเพิ่มเติม กรุณาติดต่อ:
                - Email: support@v89tech.com
                - Line: @v89tech
                - เว็บไซต์: https://v89tech.com
                """)

        # Add footer
        gr.Markdown("""
        ---

        ### 🏥 เกี่ยวกับระบบ

        **ระบบผู้ช่วยทางการแพทย์อัจฉริยะสำหรับวิเคราะห์อาการเบื้องต้นและให้คำแนะนำภาษาไทย** พัฒนาโดย:
        - 🔬 วิจัยและพัฒนา: ทีมงาน Super AI Engineer SS5
        - 🏥 ที่ปรึกษาทางการแพทย์: แพทย์ผู้เชี่ยวชาญหลายสาขา
        - 💻 เทคโนโลยี: Python, Transformers, Gradio, Multi-Modal AI

        **⚖️ ข้อปฏิเสธความรับผิดชอบ:**
        ระบบนี้ให้ข้อมูลเพื่อการศึกษาและคำแนะนำเบื้องต้นเท่านั้น ไม่ใช่การวินิจฉัยหรือรักษาทางการแพทย์
        กรุณาปรึกษาแพทย์หรือผู้เชี่ยวชาญสำหรับการรักษาที่เหมาะสม

        **🔒 ความเป็นส่วนตัว:** เราให้ความสำคัญกับการปกป้องข้อมูลส่วนบุคคลตาม พ.ร.บ. PDPA

        ** บริษัท วี89 เทคโนโลยี จำกัด | **📈เวอร์ชัน:** 2.0 - อัปเดตล่าสุด: สิงหาคม 2025
        """)

    return demo

# ========================= Main Execution =========================

def main():
    """Main function to run the medical AI system"""
    print("🚀 Starting Enhanced Thai Medical Care AI Agent...")

    try:
        # Create and launch Gradio interface
        demo = create_gradio_interface()

        # Launch with configurations for Google Colab
        demo.launch(
            server_name="0.0.0.0",  # Allow external access in Colab
            server_port=7860,       # Default port
            share=True,             # Create shareable link
            debug=True,             # Enable debug mode
            show_error=True,        # Show errors in interface
            quiet=False,            # Show startup messages
            height=1080,             # Interface height
            favicon_path=None,      # Custom favicon (optional)
            ssl_verify=False,       # Disable SSL verification for development
            app_kwargs={
                "docs_url": None,   # Disable FastAPI docs
                "redoc_url": None   # Disable ReDoc
            }
        )

    except KeyboardInterrupt:
        print("\n🛑 System stopped by user")
    except Exception as e:
        print(f"❌ Error starting system: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print("👋 Enhanced Thai Medical Care AI Agent shutdown complete")

if __name__ == "__main__":
    # Check if running in Google Colab
    try:
        import google.colab
        IN_COLAB = True
        print("📱 Running in Google Colab environment")
    except ImportError:
        IN_COLAB = False
        print("💻 Running in local environment")

    # Set environment variables for better performance
    import os
    os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Avoid tokenizer warnings
    os.environ["TRANSFORMERS_CACHE"] = "/tmp/transformers_cache"  # Cache directory

    # Run the main application
    main()